# FC Debug Steps

This notebook walks through loading HCP data, building connectivity with no normalization,
running a short BrainPy Wendling simulation, and computing FC correlation.


In [1]:
from pathlib import Path
import sys
import numpy as np
import scipy.io as sio

PROJECT_ROOT = Path.cwd().parent
sys.path.insert(0, str(PROJECT_ROOT / 'src'))

from wendling_sim import simulate
from wendling_sim.connectivity.io import load_connectivity

STRUCT_PATH = PROJECT_ROOT / 'data' / 'demo_HCP_data' / '102816' / 'structural' / 'DTI_CM.mat'
FMRI_PATH = PROJECT_ROOT / 'data' / 'demo_HCP_data' / '102816' / 'functional' / 'TC_rsfMRI_REST1_LR.mat'
TR_S = 0.72
BURN_S = 10.0


/home/changtommy16/miniconda3/envs/brainpy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def fc_matrix(ts):
    ts = np.asarray(ts, dtype=float)
    return np.corrcoef(ts, rowvar=False)

def upper_tri_flat(mat):
    idx = np.triu_indices(mat.shape[0], k=1)
    return mat[idx]

def load_target_fc(path, tr_s, burn_s=10.0):
    mat = sio.loadmat(path)
    tc = np.asarray(mat['tc'], dtype=float)
    tc = tc[:, int(burn_s / tr_s):]
    tc = tc - tc.mean(axis=1, keepdims=True)
    std = tc.std(axis=1, keepdims=True)
    std[std == 0] = 1.0
    tc = tc / std
    return fc_matrix(tc.T)

target_fc = load_target_fc(FMRI_PATH, TR_S, burn_s=BURN_S)
target_fc_flat = upper_tri_flat(target_fc)
target_fc.shape


(94, 94)

: 

## Optional: Neurolib dataset target\n\nUse this if you want to match the Neurolib HCP target (80 nodes, mean FC).

In [ ]:
# Optional: Neurolib dataset target FC (requires Neurolib dependencies)\nneurolib_root = PROJECT_ROOT / 'Neurolib_package'\nsys.path.insert(0, str(neurolib_root))\nfrom neurolib.utils.loadData import Dataset\n\nds = Dataset('hcp')\ntarget_fc_neurolib = np.mean(ds.FCs, axis=0)\nCmat_neurolib = ds.Cmat\ntarget_fc_neurolib.shape, Cmat_neurolib.shape\n

In [ ]:
net_cfg = {
    'W_path': str(STRUCT_PATH),
    'normalize': 'none',
    'G_net': 0.1,
}
conn = load_connectivity(net_cfg)
n_nodes = conn.n_nodes

sim_cfg = {
    'dt_ms': 0.5,
    'duration_ms': 2000.0,
    'jit': True,
    'integrator': 'euler',
}

model_cfg = {
    'A': 5.0,
    'B': 22.0,
    'G': 18.0,
    'p_mean': 90.0,
}
noise_cfg = {'seed': 0, 'p_sigma': 30.0}

sim_res = simulate(sim_cfg=sim_cfg, model_cfg=model_cfg, network_cfg=net_cfg, noise_cfg=noise_cfg)
lfp = sim_res.lfp
burn_samples = int(BURN_S / (sim_cfg['dt_ms'] / 1000.0))
lfp = lfp[burn_samples:] if burn_samples < lfp.shape[0] else lfp

sim_fc = fc_matrix(lfp)
sim_fc_flat = upper_tri_flat(sim_fc)

fc_corr = np.corrcoef(sim_fc_flat, target_fc_flat)[0, 1]
fc_corr
